In [2]:
# Boto3 session with credentials taken from environtment
import boto3
import os
import pandas as pd 
import json
session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_session_token=os.environ["AWS_SESSION_TOKEN"],
)
kendra_client = session.client("kendra")
lambda_client = session.client("lambda")
translate_client = session.client("translate")
print(boto3.__version__)
#os.environ["AWS_SESSION_TOKEN"]

datasources = dict(
    connect_faq = "a2d6d845-f943-4664-ad85-c4fa3ddc0820",
	connect_docs_en_v2="d74d18af-c425-4f80-b37c-2c02e0ddf425",
	connect_blogs_en_v2="d8ec0dcd-a32e-4604-923a-f58ecd5b18ce",
	#lex_v2_docs_en="fcfc2f92-107c-4c29-9ea4-e63bbac8bd45",
	connect_blogs_en_part1="ff4256c3-e0c4-42bc-a433-0f2ff0df915b"
)

1.28.30


In [40]:
def QueryKendra(index_id, query):
    response = kendra_client.query(QueryText=query, IndexId=index_id)
    return response


def RetrieveKendra(index_id, query, domain=None):
    kwargs = dict(QueryText=query, IndexId=index_id)
    if domain:
        kwargs["AttributeFilter"] = {
                "EqualsTo": {
                    "Key": "domain",
                    "Value": {
                        "StringValue": domain,
                    },
                }
            }
    response = kendra_client.retrieve(**kwargs)
    return response

In [35]:
translate_response = translate_client.translate_text(
    Text= "qué es amazon connect?",
    SourceLanguageCode="auto", TargetLanguageCode="en"
)

In [36]:
kendra_index = "f24452bd-d7f6-42bb-a5e6-25e9a826c0db"
question =  translate_response['TranslatedText']

In [41]:
#response_query = QueryKendra(kendra_index, question)
response_retrieve = RetrieveKendra(kendra_index, question, domain='Connect')

In [42]:
response_retrieve

{'QueryId': '047c3e11-8852-49a9-b38f-bec7e56495ef',
 'ResultItems': [{'Id': '047c3e11-8852-49a9-b38f-bec7e56495ef-01292b32-9452-4799-83ce-be2e68ece734',
   'DocumentId': 'https://aws.amazon.com/connect/',
   'DocumentTitle': 'Customer Service Software - Cloud Contact Center - Amazon Connect - AWS',
   'Content': 'Improve agent productivity and customer experience across voice and digital channels with the all-in-one, AI- and ML-powered contact center. Easily scale up or down to meet customer demand, with the flexibility to onboard tens of thousands of agents working from anywhere. Save up to 80% compared to traditional contact center solutions with no minimum fees, long-term commitments, or upfront license charges. How it works With Amazon Connect, you can set up a contact center in minutes that can scale to support millions of customers. Click to enlarge What is Amazon Connect (1:28) What is Amazon Connect? Save up to 80% compared to traditional contact center solutions with no minimu

In [16]:
def get_full_body(attr_list):
    full_body = None
    for attr in attr_list:
        if attr['Key'] == 'fullbody':
            full_body = attr['Value']['StringValue']
            full_body = full_body.replace("Contact Us\nEnglish\nAWS\nDocumentation\nAmazon Connect\nAdministrator Guide\nFeedback\nPreferences\n", "")
            full_body = full_body.replace("PDF\nRSS", "")
            full_body = full_body.split("\nRELATED POSTS\n")[-1]
            full_body = full_body.split("| Permalink |  Share\n")[-1]
            full_body = full_body.split("TAGS:")[0]
            full_body = full_body.split("\nDid this page help you?")[0]
            full_body = full_body.replace("}","").replace("{","").replace('"', "'")
    return full_body

documents = []
for rs in response_retrieve['ResultItems']:
    full_body = get_full_body(rs['DocumentAttributes'])
    document = {"DocumentURI": rs['DocumentURI'], "Content": full_body}
    documents.append(document)

unique_url = []
unique_documents = []
for doc in documents:
    if doc['DocumentURI'] in unique_url: continue
    else: 
        unique_url.append(doc['DocumentURI'])
        unique_documents.append(doc)

In [17]:
unique_documents

[{'DocumentURI': 'https://docs.aws.amazon.com/connect/latest/adminguide/connect-slr.html',
  'Content': "Use service-linked roles for Amazon Connect\n\nWhat are service-linked roles (SLR) and why are they important?\nAmazon Connect uses AWS Identity and Access Management (IAM) service-linked roles. A service-linked role is a unique type of IAM role that is linked directly to an Amazon Connect instance.\nService-linked roles are predefined by Amazon Connect and include all the permissions that Amazon Connect requires to call other AWS services on your behalf.\nYou need to enable service-linked roles so you can use new features in Amazon Connect, such as tagging support, the new user interface in User management and Routing profiles, and queues with CloudTrail support.\nFor information about other services that support service-linked roles, see AWS services that work with IAM and look for the services that have Yes in the Service-Linked Role column. Choose a Yes with a link to view the s

In [100]:
articles = ""
for i in range(3):
    ud = unique_documents[i]
    articles += f"Article {i}:\n\n{ud['Content']}"

len(articles.split(" "))

4420

In [101]:
from botocore.exceptions import ClientError

def call_lambda(function_arn_or_name, event):
    try:
        print(f"Intentando invocar :{function_arn_or_name}")
        response = lambda_client.invoke(
            FunctionName=function_arn_or_name,
            InvocationType="RequestResponse",
            Payload=json.dumps(event),
        )
        print ("FUCTION_RESPONSE")
        function_response = response["Payload"].read().decode("utf-8")
        print(function_response) 

        json_response = json.loads(function_response)
        print(f"\nRespuesta:{json_response}")
        json_body = json.loads(json_response["body"])
        return json_body

    except ClientError as e:
        err = e.response
        error = err
        print(err.get("Error", {}).get("Code"))
        return "Un error invocando Lambda"


In [102]:
def build_bedrock_payload(text, question):
    prompt = f"{text}\n\nBased on the above articles, answer briefly and concisely the following question:\n\n{question}"
    prompt += "\n\nGet your answer only from the articles, whitout making up new information."
    print(prompt)

    payload = {
        "body": {
            "prompt": f"Human: {prompt} Assistant:",
            "max_tokens_to_sample": 200,
            "temperature": 0.7,
            "top_k": 100,
            "top_p": 0.9,
            "stop_sequences": ["\n\nHuman"],
        },
        "modelId": "anthropic.claude-v2",
        "accept": "*/*",
        "contentType": "application/json",
    }
    print ("PAYLOAD:")
    print (payload)
    return payload

In [103]:
bedrock_payload =  build_bedrock_payload(articles, question)

Article 0:

Contact Us
English
AWS
Documentation
Amazon Lex
V2 Developer Guide
Feedback
Preferences
Amazon Chime SDK

Use the Amazon Chime SDK to add real-time audio, video, screen sharing, and messaging capabilities to your web or mobile applications. The Amazon Chime SDK provides public switched telephone network (PSTN) audio service so that you can build custom telephony applications with an AWS Lambda function.
Amazon Chime PSTN audio is integrated with Amazon Lex V2. You can use this integration to access Amazon Lex V2 bots as interactive voice response (IVR) systems in contact centers for audio interactions. Use this to integrate Amazon Lex V2 using PSTN audio services in the following scenarios.
Contact center integrations—You can use the Amazon Chime Voice Connector and Amazon Chime SDK PSTN audio service to access Amazon Lex V2 bots. Use them in any contact center application that uses the session initiation protocol (SIP) for voice communications. This integration adds natura

In [104]:
return_value = call_lambda("arn:aws:lambda:us-east-1:147654516002:function:BedrockLambda-FCallBedrock4BAEA876-A3tVXbSXt0qJ", bedrock_payload)

Intentando invocar :arn:aws:lambda:us-east-1:147654516002:function:BedrockLambda-FCallBedrock4BAEA876-A3tVXbSXt0qJ
FUCTION_RESPONSE
{"statusCode": 200, "headers": {"Content-Type": "text/html;charset=UTF-8", "charset": "UTF-8", "Access-Control-Allow-Origin": "*"}, "body": "{\"event\": {\"body\": {\"prompt\": \"Human: Article 0:\\n\\nContact Us\\nEnglish\\nAWS\\nDocumentation\\nAmazon Lex\\nV2 Developer Guide\\nFeedback\\nPreferences\\nAmazon Chime SDK\\n\\nUse the Amazon Chime SDK to add real-time audio, video, screen sharing, and messaging capabilities to your web or mobile applications. The Amazon Chime SDK provides public switched telephone network (PSTN) audio service so that you can build custom telephony applications with an AWS Lambda function.\\nAmazon Chime PSTN audio is integrated with Amazon Lex V2. You can use this integration to access Amazon Lex V2 bots as interactive voice response (IVR) systems in contact centers for audio interactions. Use this to integrate Amazon Lex V

In [105]:
print(return_value['response']['body']['completion'])

 Based on the information in Article 0 and Article 1, Amazon Connect does natively support SIP Trunk integration. 

Specifically, Article 0 mentions that "Amazon Chime PSTN audio is integrated with Amazon Lex V2. You can use this integration to access Amazon Lex V2 bots as interactive voice response (IVR) systems in contact centers for audio interactions. Use this to integrate Amazon Lex V2 using PSTN audio services in the following scenarios:

- Contact center integrations—You can use the Amazon Chime Voice Connector and Amazon Chime SDK PSTN audio service to access Amazon Lex V2 bots. Use them in any contact center application that uses the session initiation protocol (SIP) for voice communications."

And Article 1 states "When the Amazon Chime SDK sends a request to Amazon Lex V2, it includes platform-specific information to your Lambda function and conversation logs. Use this information to determine the contact center application that is sending traffic to your


In [ ]:
https://docs.aws.amazon.com/lexv2/latest/dg/contact-center.html
https://docs.aws.amazon.com/lexv2/latest/dg/bot-templates/contact-center.html